In [1]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv, set_key

In [2]:
# 当前windows
# 获取当前的 Conda 环境路径
conda_env_path = os.environ.get('CONDA_PREFIX')

# ".env" 文件的绝对路径
dotenv_path = os.path.join(conda_env_path, '.env')

# 加载 ".env" 文件
_ = load_dotenv(dotenv_path, verbose=True)
openai.api_key = os.environ['OPENAI_API_KEY']

# Introduction to Chains

# Chains and Why They Are Used

The chains are responsible for creating an end-to-end pipeline for using the language models. They will join the model, prompt, memory, parsing output, and debugging capability and provide an easy-to-use interface. A chain will 1) receive the user’s query as an input, 2) process the LLM’s response, and lastly, 3) return the output to the user.

## LLMChain

In [3]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is a word to replace the following: {word}?"

# Set the "OPENAI_API_KEY" environment variable before running following line.
llm = OpenAI(model_name="text-davinci-003", temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

In [4]:
llm_chain("artificial")

{'word': 'artificial', 'text': '\n\nSynthetic'}

In [5]:
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

llm_chain.apply(input_list)

[{'text': '\n\nSynthetic'}, {'text': '\n\nWisdom'}, {'text': '\n\nAutomaton'}]

In [6]:
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nSynthetic', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nWisdom', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nAutomaton', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 46, 'completion_tokens': 13, 'prompt_tokens': 33}, 'model_name': 'text-davinci-003'}, run=RunInfo(run_id=UUID('6e8e83d4-5352-429f-b390-42bd0d139643')))

In [7]:
prompt_template = "Looking at the context of '{context}'. What is an appropriate word to replace the following: {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=["word", "context"]))

llm_chain.predict(word="fan", context="object")
# or llm_chain.run(word="fan", context="object")

'\n\nventilator'

In [8]:
llm_chain.predict(word="fan", context="humans")
# or llm_chain.run(word="fan", context="humans")

'\n\nAdmirer'

## Parsers

In [9]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, output_parser=output_parser, input_variables=[]),
    output_parser=output_parser)

llm_chain.predict()

['Synthetic',
 'Manufactured',
 'Imitation',
 'Fabricated',
 'Fake',
 'Simulated',
 'Artificial Intelligence',
 'Automated',
 'Constructed',
 'Programmed',
 'Mechanical',
 'Processed',
 'Algorithmic',
 'Generated.']

## Conversational Chain (Memory)

In [10]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

output_parser = CommaSeparatedListOutputParser()
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

conversation.predict(input="List all possible words as substitute for 'artificial' as comma separated.")

' Synthetic, robotic, manufactured, simulated, computerized, programmed, man-made, fabricated, contrived, and artificial.'

In [11]:
conversation.predict(input="And the next 4?")

' Automated, cybernetic, mechanized, and engineered.'

## Sequential Chain

In [ ]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two])

## Debug

In [12]:
template = """List all possible words as substitute for 'artificial' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory(),
    verbose=True)

conversation.predict(input="")



> Entering new  chain...
Prompt after formatting:
List all possible words as substitute for 'artificial' as comma separated.

Current conversation:




> Finished chain.


'Synthetic, Imitation, Manufactured, Fabricated, Simulated, Fake, Artificial, Constructed, Computerized, Programmed'

## Custom Chain

In [13]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [14]:
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("artificial")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


Artificial means something that is not natural or made by humans, but rather created or produced by artificial means.

Synthetic


# Create a YouTube Video Summarizer Using Whisper and LangChain 

![图片描述](pics/youtube_video_summarizer.webp)

In [17]:
# !pip install -q yt_dlp
!pip install -q git+https://github.com/openai/whisper.git

  error: subprocess-exited-with-error
  
  git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\Administrator\AppData\Local\Temp\pip-req-build-a6zw0uj_' did not run successfully.
  exit code: 128
  
  [1 lines of output]
  fatal: unable to access 'https://github.com/openai/whisper.git/': Recv failure: Connection was reset
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\Administrator\AppData\Local\Temp\pip-req-build-a6zw0uj_' did not run successfully.
exit code: 128

See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
